## Get all NAICS Codes

In [41]:
import sys
import os 
import pandas as pd 
import bs4
import requests
import datetime
import re
import urllib
import sqlite3
import time
from tqdm.notebook import tqdm


In [134]:
base_link = "https://www.naics.com/search/"

In [135]:
page = requests.get(base_link)

In [136]:
base_soup = bs4.BeautifulSoup(page.content, 'html.parser')

In [147]:
table = base_soup.find('table')
df = pd.read_html(str(table))[0]
df_base = df.copy()
df.head()

,Code,Industry Title,Number of Business Establishments
0,11,"Agriculture, Forestry, Fishing and Hunting",378293
1,21,Mining,32231
2,22,Utilities,44408
3,23,Construction,1484279
4,31-33,Manufacturing,638730


In [138]:
table_rows = table.find_all('tr')
links = []
for tr in table_rows:
    td = tr.find_all('td')
    try:
        temp_link = td[0].find('a')['href']
        links.append(temp_link)
    except:
        pass

In [139]:
links

['https://www.naics.com/six-digit-naics/?code=11',
 'https://www.naics.com/six-digit-naics/?code=21',
 'https://www.naics.com/six-digit-naics/?code=22',
 'https://www.naics.com/six-digit-naics/?code=23',
 'https://www.naics.com/six-digit-naics/?code=31-33',
 'https://www.naics.com/six-digit-naics/?code=42',
 'https://www.naics.com/six-digit-naics/?code=44-45',
 'https://www.naics.com/six-digit-naics/?code=48-49',
 'https://www.naics.com/six-digit-naics/?code=51',
 'https://www.naics.com/six-digit-naics/?code=52',
 'https://www.naics.com/six-digit-naics/?code=53',
 'https://www.naics.com/six-digit-naics/?code=54',
 'https://www.naics.com/six-digit-naics/?code=55',
 'https://www.naics.com/six-digit-naics/?code=56',
 'https://www.naics.com/six-digit-naics/?code=61',
 'https://www.naics.com/six-digit-naics/?code=62',
 'https://www.naics.com/six-digit-naics/?code=71',
 'https://www.naics.com/six-digit-naics/?code=72',
 'https://www.naics.com/six-digit-naics/?code=81',
 'https://www.naics.co

In [140]:
df_list = []
for link in links:
    page = requests.get(link)
    temp_soup = bs4.BeautifulSoup(page.content, 'html.parser')
    table = temp_soup.find('table')
    df = pd.read_html(str(table))[0]
    df_list.append(df)

In [141]:
len(df_list)

20

In [142]:
all_naics_codes = pd.concat(df_list)
all_naics_codes['code_len'] = all_naics_codes['Codes'].apply(lambda x:len(str(x)))

In [143]:
all_naics_codes

,Codes,Titles,Total Marketable US Businesses,code_len
0,11,"Agriculture, Forestry, Fishing and Hunting",378293,2
1,1111,Oilseed and Grain Farming,61440,4
2,111110,Soybean Farming,4896,6
3,111120,Oilseed (except Soybean) Farming,208,6
4,111130,Dry Pea and Bean Farming,3666,6
...,...,...,...,...
33,9271,Space Research and Technology,138,4
34,927110,Space Research and Technology,138,6
35,9281,National Security and International Affairs,17472,4
36,928110,National Security,15308,6


In [130]:
all_naics_codes.to_csv('all_naics_codes.csv')

## Extra....

In [162]:
def expand_df(row):
    row = row[1]
    split_code = row['Code'].split('-')
    temp_min = int(split_code[0])
    temp_max = int(split_code[1])
    numb_entries = temp_max - temp_min + 1

    series_list = []
    for x in range(temp_min, temp_max+1):
        series_dict = {'Code': x, 'Industry Title': row['Industry Title']
                       , "Number of Business Establishments":row['Number of Business Establishments']/numb_entries }
        series_list.append(series_dict)
    expanded_df = pd.DataFrame(series_list)
    
    return(expanded_df)

In [165]:
def clean_industry_table(df_raw):
    df = df_raw.copy()
    df = df.dropna()
    
    df.Code = df.Code.astype('str')
    code_with_hypen_mask = df.Code.str.contains('-')
    grouped_codes = df[code_with_hypen_mask]
    
    # Expand hypen columns
    expanded_dfs = []
    for row in grouped_codes.iterrows():
        expanded_dfs.append(expand_df(row))
    df_combo = pd.concat(expanded_dfs)
    
    # Remove hypen cols 
    df_no_hypen = df[~code_with_hypen_mask]
    df_w_expanded = pd.concat([df_no_hypen, df_combo])
    df_w_expanded['Code'] = df_w_expanded['Code']
    df_w_expanded['Code'] = df_w_expanded['Code'].astype('int')
    df_w_expanded = df_w_expanded.sort_values('Code')
    return(df_w_expanded)

In [166]:
cleaned_table = clean_industry_table(df_base)

In [169]:
cleaned_table

,Code,Industry Title,Number of Business Establishments
0,11,"Agriculture, Forestry, Fishing and Hunting",378293.0
1,21,Mining,32231.0
2,22,Utilities,44408.0
3,23,Construction,1484279.0
0,31,Manufacturing,212910.0
1,32,Manufacturing,212910.0
2,33,Manufacturing,212910.0
5,42,Wholesale Trade,697579.0
0,44,Retail Trade,900083.0
1,45,Retail Trade,900083.0


In [168]:
df_alls.to_csv('naics2.csv')